In [2]:
import pandas as pd
import numpy as np

# ==========================
# 1️EXTRACT
# ==========================
file_path = "cleaned_market_data_usd.csv"
df = pd.read_csv(file_path)

print("✅ Extraction Completed!")
print(f"Shape: {df.shape}")
print("Columns:", df.columns.tolist())

# ==========================
# 2️TRANSFORM
# ==========================

# --- Step 1: Clean column names ---
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()

# --- Step 2: Convert date column ---

# --- Step 3: Remove duplicates ---
df = df.drop_duplicates()

# --- Step 4: Fill missing numeric values ---
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

# --- Step 5: Aggregation by Security, Source Feed, Vendor, Price Type, Exchange Code ---
group_cols = ["security_id", "source_feed_id", "vendor_code", "price_type", "exchange_code"]

agg_df = (
    df.groupby(group_cols)
    .agg(
        mean_price=("price", "mean"),
        median_price=("price", "median"),
        std_price=("price", "std"),
        max_price=("price", "max"),
        min_price=("price", "min"),
        count=("price", "count"),
    )
    .reset_index()
)

# --- Step 6: Merge back to full dataset ---
report = df.merge(agg_df, on=group_cols, how="left")

# --- Step 7: Calculate Price Variation (%) for each record ---
report["price_variation_percent"] = (
    abs(report["price"] - report["mean_price"]) / report["mean_price"]
) * 100

# --- Step 8: Outlier Buckets ---
def bucketize(var):
    if var < 3:
        return "<3%"
    elif 3 <= var < 5:
        return "3%-5%"
    else:
        return ">5%"

report["outlier_bucket"] = report["price_variation_percent"].apply(bucketize)

# --- Step 9: Concurrent Price Check (Max = Min = Median for the combination) ---
report["is_concurrent_price"] = (
    (report["max_price"] == report["min_price"])
    & (report["min_price"] == report["median_price"])
)

# --- Step 10: Aggregate Outlier Counts per Vendor, Price Date, Exchange, Price Type ---
outlier_summary = (
    report.groupby(["vendor_code", "date", "exchange_code", "price_type", "outlier_bucket"])
    .size()
    .reset_index(name="outlier_count")
)

# ==========================
# 3️LOAD
# ==========================

# --- Step 11: Save both detailed and summary reports ---
report.to_csv("detailed_reporting_table.csv", index=False)
outlier_summary.to_csv("vendor_performance_summary.csv", index=False)

print(" Transformation & Loading Completed Successfully!")
print(f"Detailed Report Shape: {report.shape}")
print(f"Summary Report Shape: {outlier_summary.shape}")
print(" Files Generated:")
print(" - detailed_reporting_table.csv  (Full row-level data)")
print(" - vendor_performance_summary.csv (Aggregated reporting data)")


✅ Extraction Completed!
Shape: (2041, 11)
Columns: ['Security_ID', 'Vendor_ID', 'Vendor_Code', 'Source_Feed_ID', 'Price_Type', 'Exchange_Code', 'Date', 'Currency_Code', 'Price', 'Conversion_Rate', 'Price_USD']
 Transformation & Loading Completed Successfully!
Detailed Report Shape: (2041, 20)
Summary Report Shape: (880, 6)
 Files Generated:
 - detailed_reporting_table.csv  (Full row-level data)
 - vendor_performance_summary.csv (Aggregated reporting data)
